In [ ]:
import json, glob, gc
import pandas as pd
from pandas import json_normalize

import psycopg2   
from psycopg2 import Error
import geopandas as gpd
import numpy as np
import pandas as pd

from tqdm import tqdm
from shapely.geometry import LineString, Point
from shapely.wkt import dumps, loads

from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [ ]:
table_name = ""
database = ""
user = "postgres"
password = "postgres"
port = str(5432)
host = "localhost"

In [ ]:
engine = create_engine("postgresql://"+user+":"+password+"@"+host+":"+port+"/"+database)

def generate_geometry(x):
    
    if type(x['coordinates.coordinates']) == list:
        return loads(dumps(Point(x['coordinates.coordinates'][0], x['coordinates.coordinates'][1]), rounding_precision=10))
    elif type(x['geo.bbox']) == list:
        return loads(dumps(LineString([x['geo.bbox'][:2], x['geo.bbox'][2:]]).centroid, rounding_precision=10))
    else:
        return None

In [ ]:
json_files = glob.glob('twitter_data*.json')
failed = []

for twitter_file in tqdm(range(len(json_files))):
    
    try:
        df = pd.read_json(json_files[twitter_file], lines=True)
        df = pd.concat([df, json_normalize(df['geo'])], axis=1).drop(['geo'], axis=1)
        df = df.loc[:,~df.columns.duplicated()]
        
        df = df[['id', 'text', 'created_at', 'author_id', 'source', 'author', 'place_id',
       'country', 'full_name', 'country_code', 'name', 'geo.type', 'geo.bbox',
       'coordinates.type', 'coordinates.coordinates']]
        
        df = df.drop_duplicates(subset=["text"], keep='first')

        df['geometry'] = df.apply(generate_geometry, axis=1)
        df = df[df['geometry'].notna()]

        gpd_tweets = gpd.GeoDataFrame(df, geometry='geometry')
        gpd_tweets.set_crs(epsg=4326, inplace=True, allow_override=True)
        gpd_tweets = gpd_tweets.__round__(10)

        ## Create table in postgis and insert data
        gpd.GeoDataFrame.to_postgis(gpd_tweets, name = table_name, con = engine, schema = 'public', if_exists = 'append')

        del gpd_tweets, df
        gc.collect()
    except:
        failed.append(twitter_file)
        print(len(failed))
        continue
